### `rs-client-libraries` installation

In [105]:
import rs_client
import rs_common
import rs_workflows

# Set logger level to info
import logging
rs_common.logging.Logging.level = logging.INFO

### Environment

In [106]:
import os

# In local mode, all your services are running locally.
# In hybrid or cluster mode, we use the services deployed on the RS-Server website.
# This configuration is set in an environment variable.
local_mode = (os.getenv("RSPY_LOCAL_MODE") == "1")

# In local mode, the service URLs are hardcoded in the docker-compose file
if local_mode:
    rs_server_href = None # not used
    RSPY_HOST_AUXIP = "http://localhost:8001/docs"
    RSPY_HOST_CADIP = "http://localhost:8002/docs"
    RSPY_HOST_CATALOG = "http://localhost:8003/catalog/api.html"

# In hybrid or cluster mode, they are set in an environment variables
else:
    rs_server_href = os.environ["RSPY_WEBSITE"]

In [107]:
import json
from rs_client.rs_client import RsClient
from rs_common.config import ECadipStation

# Init a generic RS-Client instance. Pass the:
#   - RS-Server website URL
#   - API key
#   - ID of the owner of the STAC catalog collections.
#     By default, this is the user login from the keycloak account, associated to the API key.
#     Or, in local mode, this is the local system username.
#     Else, your API Key must give you the rights to read/write on this catalog owner (see next cell).
#   - Logger (optional, a default one can be used)
generic_client = RsClient(rs_server_href, rs_server_api_key=None, owner_id=None, logger=None)
print(f"STAC catalog owner: {generic_client.owner_id!r}")

# From this generic instance, get an Auxip client instance
auxip_client = generic_client.get_auxip_client()

# Or get a Cadip client instance. Pass the cadip station.
cadip_station = ECadipStation.CADIP # you can also have: INS, MPS, MTI, NSG, SGS
cadip_client = generic_client.get_cadip_client(cadip_station)

# Or get a Stac client to access the catalog
stac_client = generic_client.get_stac_client()

STAC catalog owner: 'jovyan'


# Create collections and items

### Search endpoint using multiple collections + implicit naming mechanism

![title](search_img/search_demo.png)

- Possibility to specify or not the owner_id at the beginning  of the collection name or you can specify the owner_id in the filter
- First check to see if collection exist -> if not concatenate the owner_id to the collection name
- If the owner id is neither defined in the collection name nor in the filter, the default owner_id is selected

In [ ]:
# Search collection - POST method
WIDTH=2500
HEIGHT=2500

my_collections = ['S1_L1', 'S2_L1']
filter_on_dimensions = {
    "op": "and",
    "args": [
        {"op": "=", "args" : [{"property": "owner"}, "toto"]},
        {"op": "=", "args" : [{"property": "orientation"}, "nadir"]},
        {"op": "=", "args" : [{"property": "width"}, WIDTH]},
        {"op": "=", "args" : [{"property": "height"}, HEIGHT]},
    ]
}

search = stac_client.search(method='POST', filter_lang="cql2-json", filter=filter_on_dimensions, collections=my_collections)
results = list(search.items_as_dicts())
# assert results, f"No results"
# print(f"\nFound {len(results)} results for width={WIDTH} height={HEIGHT}")
for result in results:
    print(f"({result['collection']}) {result['id']}")


In [ ]:
# Search collection - GET method
filter_on_dimensions = "width=2500 AND owner='toto'"
collections = "S1_L1,toto_S2_L1"

search = stac_client.search(method='GET', filter_lang="cql2-text", filter=filter_on_dimensions, collections=my_collections)
results = list(search.items_as_dicts())
assert results, f"There should be at least one item for width={WIDTH} height={HEIGHT}"
print(f"\nFound {len(results)} results for width={WIDTH} height={HEIGHT}")
for result in results:
    print(f"({result['collection']}) {result['id']}")

In [100]:
# If owner_id is not specified -> take the default one: here "user"

# Search collection - POST method
WIDTH=2500
HEIGHT=2500

my_collections = ['S1_L1', 'S2_L1']
filter_on_dimensions = {
    "op": "and",
    "args": [
        {"op": "=", "args" : [{"property": "orientation"}, "nadir"]},
        {"op": "=", "args" : [{"property": "width"}, WIDTH]},
        {"op": "=", "args" : [{"property": "height"}, HEIGHT]},
    ]
}
try:
    search = stac_client.search(method='POST', filter_lang="cql2-json", filter=filter_on_dimensions, collections=my_collections)
    results = list(search.items_as_dicts())
    assert results, f"There should be at least one item for width={WIDTH} height={HEIGHT}"
    print(f"\nFound {len(results)} results for width={WIDTH} height={HEIGHT}")
    for result in results:
        print(f"({result['collection']}) {result['id']}")
except Exception as e:
    print(f"Error: {e}")

Error: {"error":"Collection user_S1_L1 not found."}
